# InSAR workflow

This notebook contain an example on how to execute the InSAR workflow on a local machine with SNAP installed. This workflow can produce geocoded InSAR coherence and flattened + topographic corrected + multilooked + filtered wrapped and unwrapped interferogram

In [1]:
import urllib
import json
from IPython.display import JSON

from datetime import datetime

import glob
import os
import subprocess
import shutil

## Input definition

In [2]:
burstId = 249435
sub_swath = 'IW2'
InSAR_pairs = [
    '20240809_20240821',
    '20240821_20240902'
]

# Coherence window size
cohWinRg, cohWinAz = (10, 2)

# Multillok parameters
nRgLooks, nAzLooks = (4, 1)

In [3]:
graph_interferogram = './graphs/interferogram_sarGeometry.xml'
graph_snaphu_export = './graphs/snaphu_export.xml'
graph_geocode = './graphs/geocode_snaphuInterferogram.xml'

In [4]:
if os.path.exists("CDSE_SECRET"):
    with open("CDSE_SECRET", "r") as file:
        lines = file.readlines()
    CDSE_ACCESS_KEY = lines[0].strip().split(": ")[1]
    CDSE_SECRET_KEY = lines[1].strip().split(": ")[1]
else:
    # get from environment vatiables:
    CDSE_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID", None)
    CDSE_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)


output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/InSAR_workflow'
if not os.path.exists(output_folder):
    output_folder = os.path.abspath("output")
print("output_folder: " + str(output_folder))

output_folder: /home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/InSAR_workflow


## Burst download

In [5]:
date_list = [datetime.strptime(date, "%Y%m%d") for date in list(set('_'.join(InSAR_pairs).split('_')))]

for date in date_list:

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {date.strftime('%Y-%m-%d')}T00:00:00.000Z and ContentDate/Start le {date.strftime('%Y-%m-%d')}T23:59:59.000Z and "
    f"PolarisationChannels eq 'VV' and "
    f"BurstId eq {burstId} and "
    f"SwathIdentifier eq '{sub_swath}'"
    )

    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
    bursts = json.loads(content)
    ParentProductName = bursts['value'][0]['ParentProductName']

    burst_extract_cmd = (
        f"docker run -v {output_folder}:/home/ubuntu "
        f"-e AWS_ACCESS_KEY_ID={CDSE_ACCESS_KEY} -e AWS_SECRET_ACCESS_KEY={CDSE_SECRET_KEY} "
        f"cdse_utilities sentinel1_burst_extractor.sh -o /home/ubuntu "
        f"-n {ParentProductName} -p vv -s {sub_swath.lower()} -r {burstId}"
    )

    #print(burst_extract_cmd)
    os.system(burst_extract_cmd)
    

Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.


## Sentinel-1 InSAR processing

In [5]:
for pair in InSAR_pairs:

    # Create the interferogram
    prm_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[0]}*', 'manifest.safe'))[0]
    sec_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[1]}*', 'manifest.safe'))[0]
    tmp_folder = os.path.join(output_folder, 'tmp')
    os.makedirs(tmp_folder, exist_ok=True)
    phase_filename = os.path.join(tmp_folder, f'phase_coh_{pair}.dim')
    cmd_interferogram = (
        f'gpt "{graph_interferogram}" -Pprm_filename="{prm_filename}" -Psec_filename="{sec_filename}" '
        f'-PcohWinRg={cohWinRg} -PcohWinAz={cohWinAz} '
        f'-PnRgLooks={nRgLooks} -PnAzLooks={nAzLooks} '
        f'-Poutput_filename="{phase_filename}" '
        f'-c 8192M'
    )
    os.system(cmd_interferogram)

    # Prepare the snaphu export for unwrapping
    cmd_snaphu_export = (
        f'gpt "{graph_snaphu_export}" -Pphase_filename="{phase_filename}" -Poutput_folder_snaphu="{tmp_folder}" '
    )
    os.system(cmd_snaphu_export)

    # Unwrapping with snaphu
    snaphu_conf_filename = glob.glob(os.path.join(tmp_folder, '**/snaphu.conf'))[0]
    with open(snaphu_conf_filename, 'r') as file:
        for line in file:
            if line.startswith('#'):
                line = line[1:].lstrip()  # Remove the '#' symbol and whitespaces at the beginning
                if line.startswith('snaphu'):
                    cmd_unwrapping = line.rstrip()
                    break
    work_dir = os.getcwd()
    os.chdir(os.path.dirname(snaphu_conf_filename))
    os.system(cmd_unwrapping)
    os.chdir(work_dir)

    # Geocode the result (interferogram, unwrapped interferogram, coherence)
    prm_date, sec_date = [datetime.strptime(date, "%Y%m%d") for date in pair.split('_')]
    phase_bandname = f'Phase_ifg_{sub_swath}_VV_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    unw_phase_bandname = f'Unw_Phase_ifg_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    coh_bandname = f'coh_{sub_swath}_VV_{prm_date.strftime("%d%b%Y")}_{sec_date.strftime("%d%b%Y")}'
    unw_phase_filename = glob.glob(os.path.join(tmp_folder, '**/UnwPhase*.img'))[0]
    cmd_geocode = (
        f'gpt "{graph_geocode}" -Pinterferogram_filename="{phase_filename}" -Punw_interferogram_filename="{unw_phase_filename}" '
        f'-Pphase_coh_bandnames="{phase_bandname},{unw_phase_bandname},{coh_bandname}" '
        f'-Poutput_filename="{os.path.join(output_folder, f'geocoded_interferogram_{pair}.tif')}"'
    )
    os.system(cmd_geocode)
    shutil.rmtree(tmp_folder)
    

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph


1257 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
1257 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
1264 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
1265 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.


....11%....21%....31%....41%....52%....62%....72%....82%.... done.


-- org.jblas INFO Deleting /tmp/jblas4844068873431598342/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas4844068873431598342/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas4844068873431598342/libgfortran-5.so
-- org.jblas INFO Deleting /tmp/jblas4844068873431598342/libquadmath-0.so
-- org.jblas INFO Deleting /tmp/jblas4844068873431598342
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


...

Assembling tiles
Running optimizer for secondary network
Number of nodes in secondary network: 30452
Flow increment: 1  (Total improvements: 0)
Treesize: 30452      Pivots: 8427        Improvements: 233        
Flow increment: 2  (Total improvements: 233)
Treesize: 30452      Pivots: 5           Improvements: 0          
Flow increment: 3  (Total improvements: 233)
Treesize: 30452      Pivots: 0           Improvements: 0          
Flow increment: 4  (Total improvements: 233)
Treesize: 30452      Pivots: 0           Improvements: 0          
Integrating secondary flows
Output written to file UnwPhase_ifg_IW2_VV_21Aug2024_02Sep2024.snaphu.img
Removing temporary directory snaphu_tiles_63844
SUGGESTION: Try increasing tile overlap and/or size if solution has edge artifacts
Program snaphu done
Elapsed processor time:   0:10:21.54
Elapsed wall clock time:  0:03:14


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


Executing processing graph


1031 [main] INFO hdf.hdflib.HDFLibrary - HDF4 library: 
1032 [main] INFO hdf.hdflib.HDFLibrary -  successfully loaded.
1035 [main] INFO hdf.hdf5lib.H5 - HDF5 library: 
1035 [main] INFO hdf.hdf5lib.H5 -  successfully loaded.


....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
